<a href="https://colab.research.google.com/github/Vicks-2019/AutoML/blob/main/Movie_reccomendation_using_TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

movies_raw = pd.read_csv('https://raw.githubusercontent.com/bialesdaniel/se4ai-i5-tpot/master/movie-data.csv')
users_raw = pd.read_csv('https://raw.githubusercontent.com/bialesdaniel/se4ai-i5-tpot/master/users-data.csv')
ratings_raw = pd.read_csv('https://raw.githubusercontent.com/bialesdaniel/se4ai-i5-tpot/master/ratings-data.csv')

users_raw.set_index('user_id', inplace=True)
movies_raw.set_index('movie_id', inplace=True)

In [2]:
#Movie data
movies_raw.head()

,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,tmdb_id,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,,,
if+you+could+only+cook+1935,0.0,"['Comedy', 'Romance']",tt0026519,en,If You Could Only Cook,An auto engineer (Herbert Marshall) and a prof...,0.430813,['Columbia Pictures'],['United States of America'],1935-12-30,0.0,72,['English'],If You Could Only Cook,126083,6.8,4
the+tattooed+widow+1998,0.0,[],tt0120299,sv,Den tatuerade änkan,Ester is the perfect grandmother everyone expe...,0.012333,[],[],1998-03-10,0.0,0,[],The Tattooed Widow,80059,0.0,0
pride+and+prejudice+1980,0.0,['Drama'],tt0078672,en,Pride and Prejudice,Mrs. Bennet is determined to find husbands for...,0.073979,"['Australian Broadcasting Corporation', 'Briti...","['United Kingdom', 'Australia']",1980-01-13,0.0,265,['English'],Pride and Prejudice,77172,6.8,14
flodder+1986,0.0,['Comedy'],tt0091060,fr,Flodder,A low-class a-social family ends up in a rich ...,6.702840,['First Floor Features'],['Netherlands'],1986-12-17,0.0,111,['Nederlands'],Flodder,10570,6.6,32
cold+turkey+1971,0.0,['Comedy'],tt0066927,en,Cold Turkey,Reverend Brooks leads the town in a contest to...,0.609080,"['Tandem Productions', 'DFI']",['United States of America'],1971-02-19,11000000.0,99,['English'],Cold Turkey,42493,7.5,4


In [3]:
#User data
users_raw.head()

,age,gender,occupation
user_id,,,
800720,42,M,scientist
577694,31,M,college/grad student
207937,28,F,college/grad student
584780,33,M,executive/managerial
149240,34,F,sales/marketing


In [4]:
#Rating data
ratings_raw.head()

,movie_id,rating,user_id
0,jackass+the+movie+2002,4,615847
1,mission+impossible+iii+2006,4,284854
2,good+will+hunting+1997,5,792329
3,happy+gilmore+1996,3,709941
4,moulin+rouge+2001,3,531076


In [5]:
#Data Preprocessing
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer

class MultiLabelStringToArray(BaseEstimator, TransformerMixin):
    """
    This shapes the data to be passed into the MultiLabelBinarizer. It takes
    columns that are array-like strings and turns them into arrays.
    """
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = X.copy()
        for column_name in df.columns:
            df[column_name] = self._transform_column_to_array(df[column_name])
        return df

    def _transform_column_to_array(self, pd_column):
        transformed_column = pd_column.copy()
        
        # replace null cells with empty array
        transformed_column.loc[transformed_column.isnull()] = transformed_column.loc[
            transformed_column.isnull()
        ].apply(lambda x: '[]')

        # parse string into array
        transformed_column = transformed_column.apply(self._parse_arraystr)
        return transformed_column

    def _parse_arraystr(self, str):
        """
        Applies a number of rules to turn an array looking string into an array
          - remove brackets
          - remove quotes
          - remove extra spaces
          - deliminate by comma
          - remove empty string entries in the array
        """
        str_without_brackets = str.replace("[","").replace("]","")
        str_without_quotes = str_without_brackets.replace("'","")
        str_without_spaces = str_without_quotes.replace(" ","")
        list_with_empties = str_without_spaces.split(',')
        if '' in list_with_empties:
            while("" in list_with_empties) : 
                list_with_empties.remove("") 
        return np.array(list_with_empties)
    
class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    """
    This tranformer creates a MultiLabelBinarizer for every column passed in.
    """
    def __init__(self):
        self.mlbs = {}
    def fit(self, X, y=None):
        """Fit the MultiLabelBinarizer to the data passed in"""
        df = X.copy()
        for column_name in df.columns:
            mlb = MultiLabelBinarizer()
            mlb.fit(df[column_name])
            # Uncomment the following line if you want to print out the values
            # that the MultiLabelbinarizer discovered.
            #print('Column: {NAME} Values: {VALUES}'.format(NAME=column_name, VALUES=mlb.classes_))
            self.mlbs[column_name] = mlb
        return self
    def transform(self, X, y=None):
        """
        Returns a dataframe with the binarized columns. When applied in a
        ColumnTransformer this will effectively remove the original column and 
        replace it with the binary columns
        """
        df = X.copy()
        binarized_cols = pd.DataFrame()
        for column_name in df.columns:
            mlb = self.mlbs.get(column_name)
            new_cols = pd.DataFrame(mlb.transform(df[column_name]),columns=mlb.classes_)
            binarized_cols = pd.concat([binarized_cols, new_cols], axis=1)
        return binarized_cols

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class ExtractReleaseDateFeatures(BaseEstimator, TransformerMixin):
    """
    This transformer takes a column with a date string formatted as 
    'YYYY-mm-dd', extracts the year and month, and returns a DataFrame with
    those columns.
    """
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        """
        Returns a dataframe with the year and month as integer fields. When 
        applied in a ColumnTransformer this will effectively remove the
        original column and replace it with the new columns.
        """
        df = X.copy()

        # fill nulls values that wont show up in valid data
        df = df.fillna('0000-00-00') 

        df['year'] = df.iloc[:,0].apply(lambda x: str(x)[:4])
        df['month'] = df.iloc[:,0].apply(lambda x: str(x)[5:7])
        df = df.astype({'year':'int64', 'month':'int64'})

        return df.loc[:,['year','month']]

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pipeline to create binary columns
multilabel_binarizer_pipeline = Pipeline([
    ('multilabel_str_to_array',MultiLabelStringToArray()),
    ('binarizer', MultiLabelBinarizerTransformer()),
],verbose=True)

MULTILABEL_BINARIZER_COLUMNS = ['genres','production_countries', 'spoken_languages', 'gender', 'occupation']
RELEASE_DATE_COLUMNS = ['release_date']
PASSTHROUGH_COLUMNS = ['age','budget','popularity','revenue', 'runtime', 'vote_average', 'vote_count']

full_data_clean_pipeline = ColumnTransformer([
    ('multilabel_binarizer', multilabel_binarizer_pipeline, MULTILABEL_BINARIZER_COLUMNS),
    ('release_date', ExtractReleaseDateFeatures(), RELEASE_DATE_COLUMNS),
    ('passthrough_columns','passthrough', PASSTHROUGH_COLUMNS)
],remainder='drop',verbose=True)

In [8]:
records = ratings_raw.join(users_raw, on='user_id', how='left')
records = records.join(movies_raw, on='movie_id', how='left')

records = records.sort_values(by=['user_id'])
records = records.dropna(subset=['budget','popularity','revenue','runtime','vote_average','vote_count'])
records.head()

,movie_id,rating,user_id,age,gender,occupation,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,tmdb_id,vote_average,vote_count
18981,legends+of+the+fall+1994,5,8,30,M,college/grad student,30000000.0,"['Adventure', 'Drama', 'Romance', 'War']",tt0110322,en,Legends of the Fall,An epic tale of three brothers and their fathe...,12.19900,"['Bedford Falls Productions', 'TriStar Picture...",['United States of America'],1994-12-16,160639000.0,133.0,"['', 'English']",Legends of the Fall,4476.0,7.2,636.0
18869,miracle+on+34th+street+1994,4,8,30,M,college/grad student,0.0,"['Fantasy', 'Drama', 'Family']",tt0110527,en,Miracle on 34th Street,A little girl discovers dreams can come true i...,5.83162,['Twentieth Century Fox Film Corporation'],['United States of America'],1994-11-18,46264400.0,114.0,['English'],Miracle on 34th Street,10510.0,6.4,199.0
20104,four+weddings+and+a+funeral+1994,4,8,30,M,college/grad student,6000000.0,"['Comedy', 'Drama', 'Romance']",tt0109831,en,Four Weddings and a Funeral,Four Weddings And A Funeral is a British comed...,8.99035,"['Channel Four Films', 'PolyGram Filmed Entert...",['United Kingdom'],1994-03-09,254701000.0,117.0,['English'],Four Weddings and a Funeral,712.0,6.6,654.0
20117,jurassic+park+1993,4,8,30,M,college/grad student,63000000.0,"['Adventure', 'Science Fiction']",tt0107290,en,Jurassic Park,A wealthy entrepreneur secretly creates a them...,8.86378,"['Universal Pictures', 'Amblin Entertainment']",['United States of America'],1993-06-11,920100000.0,127.0,"['English', 'Español']",Jurassic Park,329.0,7.6,4956.0
17547,braveheart+1995,5,8,30,M,college/grad student,72000000.0,"['Action', 'Drama', 'History', 'War']",tt0112573,en,Braveheart,"Enraged at the slaughter of Murron, his new br...",20.75510,"['Icon Entertainment International', 'The Ladd...",['United States of America'],1995-05-24,210000000.0,177.0,"['English', 'Français', 'Latin', '']",Braveheart,197.0,7.7,3404.0


In [9]:
pip install tpot xgboost

     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 163kB 13.0MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=6adcebe2c2dda238ea9948902b0038f08d75ab8422b2f5ef12c557bd82db2291
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [10]:
X_all = full_data_clean_pipeline.fit_transform(records)
y_all = records['rating']

X_train = X_all[:10000]
y_train = y_all[:10000]
X_test = X_all[10000:20000]
y_test = y_all[10000:20000]

[Pipeline]  (step 1 of 2) Processing multilabel_str_to_array, total=   8.5s
[Pipeline] ......... (step 2 of 2) Processing binarizer, total=  15.4s
[ColumnTransformer]  (1 of 3) Processing multilabel_binarizer, total=  24.7s
[ColumnTransformer] .. (2 of 3) Processing release_date, total=   0.6s
[ColumnTransformer]  (3 of 3) Processing passthrough_columns, total=   0.0s


In [11]:
from tpot import TPOTRegressor

pipeline_optimizer = TPOTRegressor(generations=10, population_size=10, verbosity=2, random_state=42,
                                   template='Selector-Transformer-Regressor', n_jobs=-1,
                                   warm_start=True, periodic_checkpoint_folder='/content/tpot-intermediate-save/')
pipeline_optimizer.fit(X_train, y_train)


Generation 1 - Current best internal CV score: -0.9498486944780838

Generation 2 - Current best internal CV score: -0.9498486944780838

Generation 3 - Current best internal CV score: -0.9498486944780838

Generation 4 - Current best internal CV score: -0.9498486944780838

Generation 5 - Current best internal CV score: -0.9490789352572977

Generation 6 - Current best internal CV score: -0.9490789352572977

Generation 7 - Current best internal CV score: -0.9490789352572977

Generation 8 - Current best internal CV score: -0.9490789352572977

Generation 9 - Current best internal CV score: -0.9490789352572977

Generation 10 - Current best internal CV score: -0.9489623748375162

Best pipeline: LassoLarsCV(PCA(SelectPercentile(input_matrix, percentile=56), iterated_power=8, svd_solver=randomized), normalize=False)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=10,
              log_file=None, max_eval_time_mins=5, max_time_mins=None,
              memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder='/content/tpot-intermediate-save/',
              population_size=10, random_state=42, scoring=None, subsample=1.0,
              template='Selector-Transformer-Regressor', use_dask=False,
              verbosity=2, warm_start=True)

In [14]:
pipeline_optimizer.score(X_test, y_test)

-0.9945632929508181

In [15]:
X_train_large = X_all[:500000]
y_train_large = y_all[:500000]
X_test_large = X_all[500000:]
y_test_large = y_all[500000:]

In [ ]:
from tpot import TPOTRegressor

pipeline_optimizer_large = TPOTRegressor(generations=10, population_size=10, verbosity=2, random_state=42,
                                      template='Selector-Transformer-Regressor', config_dict="TPOT light", n_jobs=-1,
                                     warm_start=True, periodic_checkpoint_folder='/content/tpot-intermediate-save-large/')
pipeline_optimizer_large.fit(X_train_large, y_train_large)


Generation 1 - Current best internal CV score: -0.9792904711514234

Generation 2 - Current best internal CV score: -0.9792904711514234

Generation 3 - Current best internal CV score: -0.9792904711514234

Generation 4 - Current best internal CV score: -0.9792904711514234

Generation 5 - Current best internal CV score: -0.9792904711514234

Generation 6 - Current best internal CV score: -0.9747907575781778

Generation 7 - Current best internal CV score: -0.9648974507933257


In [ ]:
pipeline_optimizer_large.score(X_test_large, y_test_large)